# The Poisson Process

In this exercise we are going to look at the Poisson process as an example of a Markov chain in continuous time.  As such before doing this exercise remind yourself of the theory of Markov chains in continuous time by watching this video https://www.youtube.com/watch?v=tbA2DnKTRxM.  You should remind yourself about the Poisson process by watching this video https://www.youtube.com/watch?v=kWvG0_p4wO8.  Once you have revised all this material put the following items in your notes on this exercise:

- Write out the Kolmogorov equation.
- Write out the transition graph for the Poisson Process
- Write out the jump rate matrix for the Poisson Process
- Write (but don't proove) an expression for the probability having $n$ events from a Poisson process over a time window of length $t$.

Now load the code in the following window, which we will use for the problems in the remainder of this exercise.

In [ ]:
import math
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object) :
    def __init__( self, ngen ):
        self.ngen = ngen 
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0, self.ngen), ylim=(0, 0.1))
        
    def setup(self):
        self.xdata, self.ydata = [], []
        self.line, = self.ax.plot([],[],'.')
        return self.line,
    
    def run( self, data ):
        times,y = data
        for t in times :
            self.xdata.append(t)
            if y<0 :
                self.ydata.append(0.05)
            else :
                self.ydata.append( len(self.xdata) )
        self.ax.set_xlim( 0, self.xdata[-1] )
        if y>0 :
            self.ax.set_ylim( 0, self.ydata[-1] )
        self.line.set_data( self.xdata, self.ydata )
        return self.line,
    
def dynamicplot( ngen, operation ):
    myplot = plotobj( ngen )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen ), 
                                init_func=myplot.setup, interval=100, blit=False, repeat=False)

# Generating a Poisson process


Writing a program to generate a Poisson process is more complicated than generating a process to study discrete-time Markov chains.  The additional difficulty with the Poisson process comes because the events arrive at random times.  Remember that in discrete-time Markov chains the random events (the transitions) between states happen at fixed points in time.  Regardless of this difficulty, however, it is still possible to write a program to generate events according to a Poisson process.  An example code that does this is given in the cell below.

In [ ]:
# Function takes as arguments the start time, the lambda parameter for the Poisson random variable 
# and the length of a random variable
def genpoisson( startt, param, time ):
    # Generate a uniform random variable betweem zero and one
    rand = random.uniform(0,1)
    N=0
    # Calculate the probability that no events happen during the time window
    p=math.exp( -param*time )
    F=p
    # This loop is used to calculate the number of events during our time window
    while rand>F : 
        N = N + 1
        # Calculate the probability that N events happen during the time window
        p = param*time*p/float(N)
        F = F+p
    # I have now generated a random variable that tells me the number of events 
    # that occur during the time window     
        
    times = []
    # Now we find the times at which the random events occur.  Essentially these 
    # occur at random times during our time window of length time.
    for i in range(0,N) :  
        times.append( startt + time*random.uniform(0,1) )
    # This just sorts the list of times so that the early events are first and 
    # later events are later in the array
    times = sorted(times)
    # Return the number of events that happen during the time window (N) 
    # and the times at which these events occured.
    return N, times

The above code works by recognising that probablility, $p_n$, that $n$ events occur in a time window of length $t$ is given by the following expression:

$$
p_n = \frac{(\lambda t)^n}{n!} e^{-\lambda t}
$$

where $\lambda$ is the rate parameter.  This expression is just the probability mass function for a Poisson Random Variable with parameter $\lambda t$.  We can thus generate a Poisson process by generating random numbers from the Poisson distribution.  To generate a random variable from a Poisson random variable we can exploit a technique that we already saw in the first of the programming exercises on discrete Markov chains.  If you remember in that video I showed you how if we have a random variable, $X$, with the following probability mass function:

$$
f_X(x) = 
\begin{cases}
p_0 & \textrm{if} \quad x=0 \\
p_1 & \textrm{if} \quad x=1 \\
p_2 & \textrm{if} \quad x=2 \\
0 & \textrm{otherwise}
\end{cases}
$$

We can do this by instead generating a uniform random variable, $Y$, between 0 and 1.  We then generate our random variable $X$ using:

$$
X = \begin{cases}
0 & \textrm{if} \quad 0 < Y < p_0 \\
1 & \textrm{if} \quad p_0 < Y < p_0+p_1 \\
2 & \textrm{if} \quad p_0+p_1 < Y < 1 \\
\end{cases}
$$

We have seen this trick before in the programming exercise on the Markov chain where we had to generate a random variable between 1 and 3 and where the probabilities of gettting 1, 2 and 3 were all different.  In the code above I am exploiting the exact same trick to calculate the Poisson random variable, $Z$, that tells me the number of events that occur in the time window of length $t$.  The only difference now is that $Z$ can take an infinite number of values.  I thus get $Z$ from a uniform random variable, $Y$, between 0 and 1 by using something like the following:

$$
Z = \begin{cases}
0 & \textrm{if} \quad 0 < Y < e^{-\lambda t} \\
1 & \textrm{if} \quad e^{-\lambda t} < Y < e^{-\lambda t} + \lambda t e^{-\lambda t} \\
\vdots & \vdots \\
N & \textrm{if} \quad \sum_{k=0}^{N-1} \frac{(\lambda t)^k}{k!} e^{-\lambda t} < Y < \sum_{k=0}^{N} \frac{(\lambda t)^k}{k!} e^{-\lambda t} \\
\vdots & \vdots 
\end{cases}
$$

Read this first part carefully and take some time now to look at the code and understand how this is implemented.  The part of the code above that does this part finishes with a comment that reads "I have now generated a random variable that tells me the number of events that occur during the time window."

Once I have found out how many events occur during my time window of length $t$ I next have to work out the times at which these events occur.   To do this I exploit a derivation involving the following conditional probability distribution:

$$
P(T \le t | T \le \tau ) \qquad \textrm{where} t < \tau
$$

In words this is the probability that an event occured before time $t$ given that it definitely occured before time $\tau$.  Furthermore, in order to complete this proof we need to use the fact that if the event occurs before time $t$ it will definitely <b> not </b> occur in the time window between time $t$ and time $\tau$.  The probability in the numerator of the following conditional probability can thus be calculated using elements from the probability mass function for the Poisson process.  The probability in the denominator can be calculated in a similar way.  We can thus arrive at:

$$
\begin{aligned}
P(T \le t | T \le \tau ) & = \frac{P(T \le t) P(\textrm{no events between $t$ and $\tau$})}{P(T\le \tau)} \\
 & =\frac{ \lambda t e^{-\lambda t} e^{-\lambda( \tau-t) } }{ \lambda \tau e^{-\lambda \tau} } \\
 & = \frac{ t e^{-\lambda( t+ \tau - t)}}{\tau e^{-\lambda \tau}} \\
 & = \frac{ t e^{-\lambda \tau} }{ \tau e^{-\lambda \tau} } \\
 P(T \le t | T \le \tau ) & = \frac{t}{\tau}
 \end{aligned}
$$

In other words, if we know an event occurs in a time window of length $t$ the exact time at which the event occurs is given by a uniform random variable.  Furthermore, as there is no correlation between the timings of the events in the Poisson process if $N$ events occur during our time window they will all occur at random times during that time interval. 

This mathematics is exploited in the second half of the function above - the part the calculates the timings of the random variables.  Remember that in the first part of the function we generated a random variable to tell us the number of events, $N$, that occured during the time interval.  In the second part of the code we thus generate the times at which these events occur by generating $N$ random variables between $0 and $t$ - where $t$ is the length of our time interval.  The final two lines of the code then sort these random variables into ascending order as this is useful in terms of the activities that we will do in the later parts of this exercise.  Read the code above again now and make sure that you understand what it is doing before proceding with the remainder of this exercise.

# Visualizing the Poisson process

In the cell below I have written a function that calls the genpoisson function that we have just examined in detail multiple times.  By doing this I am able to simulate the Poisson process for some time.  I then use a call to one of the usual dynamic plotting routines to visualize an instance of the Poisson process.  If you thus press shift and enter on the cell below you should get a plot which will show you the times at which the (random) events in the Poisson Process we are simulating occured on the time axis.  The timing of each event in the process is shown as a blue dot in this figure.  The time at which the event occured is shown on the x-axis.  The y-axis has no meaning in this plot.  Run the code below now and in your notes describe what you observe about the distribution of events.

In [ ]:
def genprocess( ngen ) :
    time = 0
    for i in range(0,ngen) :
        time += 2
        N, times = genpoisson( time-2, 3, 2 )
        yield times, -1

# The first number here controls the number of times genpoisson is called.
dynamicplot( 5, genprocess )

Lets now do a second exeriment on the Poisson process.  Lets look at the number of events that occur in each of our time windows.  The code in the cell below will call the genpoisson code from above 10 times and will then plot the number of events that occured in each time window on a graph.  Before you run this code think about whether you expect the number of events in each time window to be the same or would you expect these values to be different?

In [ ]:
xdata,ydata = [],[]
for i in range(0,10):
    N,times = genpoisson( 0, 3, 10 )
    xdata.append(i)
    ydata.append(N)
    
fig = plt.figure()
ax = plt.axes()
ax.bar(xdata, ydata, 0.8)
fig.show()

You should have seen from this exercise that the number of events in each of the time windows is not the same.  This makes a lot of sense as the number of events that occur in any given time window is given by a Poisson Random variable.

Lets now turn to how the total number of events that have occured changes as a function of time.  The code in the cell belows again calls the genpoisson function multiple times.  This time, however, it will plot a graph showing how the total number of events changes (y-axis) as a function of time (x-axis).  Run this code now by press shift enter on the cell below.  What (rough) shape does this curve have? 

In [ ]:
def genprocess2( ngen ) :
    time, tot = 0, 0
    for i in range(0,ngen) :
        time += 2
        N, times = genpoisson( time-2, 3, 2 )
        yield times, N
        
# The first number here is the number of times genpoisson is called.  This is the number 
# you should reduce when you are called to in the exercise.
dynamicplot( 100, genprocess2 )

The curve you get out is rougly linear.  If you look more closely the curve is not truly linear, however.  (This becomes even clearer if you run the code again calling the genpoisson function fewer so try this now.)  <b> In your notes explain why the curve is not exactly linear and explain what the gradient of the line of best fit to the curve should rougly be equal to</b>    

# Inter-arrival times

In this next exercise we are going to look at the probability distribution for the timings between the events in our Poisson process.  I have written some of the code that you will need to do this in the cell below.   You will need to fill in the blanks, however.  In order to understand what you have to do lets go through what the code below does:

- The first part of the code calls the functiog genpoisson 1000 times.  Each time the genpoisson process is called we simulate the poisson process for three seconds.  By calling genpoisson 1000 times we thus simulate the Poisson process for 3000 seconds.  The times at which the events occur during this period are stored in an array called mylist.

- We are going to use the next part of the code to calculate the cumulative probability distribution function for the random variable $T$ that measures the time between events in our Poisson process.  We will do this by creating a number of auxiliary (Bernouli) random variables using the following formula:

$$
Y_t = \begin{cases}
1 & \textrm{if} \quad T \le t \\
0 & \textrm{otherwise}
\end{cases}
$$

By running our Poisson process for a long time we have generate a large number of samples of the random variable $T$ and by extension we have generated a large number of samples for our random variables $Y_t$.  We can thus estimate the $\mathbb{E}(Y_t)$ values by exploiting the law of large numbers (as we have done in many of the other programming exercises in this course).  This is what I would like you to implement in the code below.  I have started you off by creating a list full of zeros (histo) that you can use to store the sums.  I have also provided a list (tval) of the various values that you should use to for the small superscript $t$ when estimating the above expectation values.  Remember also that you will need to normalise your esimtate of the probability distribution before plotting it.

- The last part of the code below will plot your estimate of the cumulative probability distribution function.  You will need to calculate the approximate distribution function before you are able to see it, however.  What shape does this cumulative probability distribution have?  Hence, what type of random variable tells you about the distribution of times between events in a Poisson process?  

In [ ]:
# This first part simulates the Poisson process for 
# 3000 s and stores the timings of the random events
# in a list called mylist
param, window = 2, 3
time, mylist=0, []
for i in range(0,1000) :
    N, times = genpoisson( time, param, window )
    time += window
    for t in times :
        mylist.append( t )

# This second part generates all the variables you will
# need to calculate your (approximate) cumulative proability
# distribution function.  tval holds the small underscore t
# values for the formula in the above while histo will eventually
# hold your esimtates for the E(Y_t) values.
tmin, tmax, nbins  = 0.0, 4.0, 100
delr = (tmax - tmin ) / float(nbins)
tval, histo = [], []
for i in range(0,nbins) :
    tval.append( xmin+float(i)*delr )
    histo.append(0)
    
# You will need to put your code to calcualte the probability distribution
# function here.


# This part plots your esimtate of the probability distribution function
fig = plt.figure()
ax = plt.axes( xlim=(xmin,xmax), ylim=(0,1) )
ax.plot(tval, histo, ".")

# An alternative way to generate Poisson random variables

In the previous exercise we showed that the times between the random events in a Poisson process are distributed according to the exponential distribution.  In other words, the time interval between the $N$th event and the $(N+1)$th event in a poisson process is given by an exponential distributed random variable.  This is a very useful fact as it gives us an alternative method for simulating the Poisson random variable.  Furthermore, with this realisation in place we can create new types of random processes by simply changing the random variable that controls the timings between events.  We are going to explore these ideas in this final exercise but first I have written some code below in order to allow you to plot these random processes.  Make sure you press shift and enter on the cell below before continuing on with these exercises. 

In [ ]:
class plotobj2(object) :
    def __init__( self, ngen ):
        self.ngen = ngen 
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0, self.ngen), ylim=(0, 0.1))
        
    def setup(self):
        self.xdata, self.ydata = [], []
        self.line, = self.ax.plot([],[],'.')
        return self.line,
    
    def run( self, data ):
        t = data
        self.xdata.append(t)
        self.ydata.append( len(self.xdata) )
        self.ax.set_xlim( 0, self.xdata[-1] )
        self.ax.set_ylim( 0, self.ydata[-1] )
        self.line.set_data( self.xdata, self.ydata )
        return self.line,
    
def raw( ngen, time, myvar ):
    cnt = 0
    while cnt < ngen :
        cnt += 1
        time = myvar(time)
        yield time
    
def dynamicplot2( ngen, myvar ):
    time, myplot = 0, plotobj2( ngen )
    return  anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=raw( ngen, time, myvar ), 
                                init_func=myplot.setup, interval=100, blit=False, repeat=False)

The code below contains eventhing else we need for our new way of simulating the Poisson process.  Lets quickly discuss how this code works.  The argument to this function (Tn) is the time of the last event.  In the equations that follow we will call this time $T_n$.  We know (from what we have learnt in the previous exercises that the time until the next event is now a exponentially-distributed random variable, $W$.  We can thus calculate the time, $T_{n+1}$, at which the next event will occur using:

$$
T_{n+1} = T_n + W
$$

Consequently, all we need to do is generate the exponentially distributed random variable $W$ and add it to the input variable Tn or $T_n$ in the above mathematical equations.  We can generate this exponentially distributed random variable by using the continuous analogue of the equation that we used to generate Poisson distributed random variables in the earlier exercises.  If you look at the equations in the above they essentially boiled down to calcualting a uniformly distributed random variable, $U$, and generating an integer random variable, $N$, in accordance with the following rule:

$$
N = n \qquad \textrm{if} \quad \sum_{i=0}^{n-1} f_N(i) < U < \sum_{i=0}^{n} f_N(i)
$$

where in these expressions $f_N(i)$ is used to denote the probability mass function for the discrete random variable $N$.  In the continuous analogue of this expression the sums are replaced by integrals and the probability mass functions are replaced by probability densities.  We thus have that we can generate a continuous random variable, X, that has probability mass function $f_X(x)$ from a uniform random variable $U$ that lies between 0 and 1 using:

$$
X = x \qquad \textrm{if}\quad \int_{-\infty}^{x-\epsilon} f_X(x') \textrm{d}x' < U < \int_{-\infty}^{x} f_X(x') \textrm{d}x'
$$

In these expressions $\epsilon$ is a very small number.  Furthermore, notice that the two integrals are just the cumulative probability distribution function for the random variable $F_X(x)$.  We can thus rewrite the above as:

$$
X = x \qquad \textrm{if}\quad F_X(x-\epsilon) < U < F_X(x)
$$

In addition, because $\epsilon$ is a small number we can assume that the range of valid values for $U$ after the if is essentially zero.  We can thus rewrite this expression as follows:

$$
X = F_X^{-1}(U)
$$

where $F_X^{-1}(x)$ is the inverse function for the cumulative probability distribution function $F_X(x)$.  <b> In your notes calcualte this inverse function for the exponential random variable.</b>  Remember that the probability mass function for the exponential random variable is:

$$
F_X(x) = 1 - e^{-\lambda t}
$$

<b> Explain why the code below works based on your derivation and give the value that I have used for $\lambda$ within the code. </b>  Once you are done press shift and enter on the cell below.  When you have done so a plot of the number of events as a function of time will be shown.  As we have see before you should note that the slope of the above line is nearly linear but that there are deviations around this linear behavior because of the random nature of the process. 

In [ ]:
def poisson2( Tn ) :
    rand = random.uniform(0,1)
    return Tn - math.log( rand ) / 2.0

dynamicplot2( 100, poisson2 )

# Semi-Markov processes 

The realization that the timings between the events in a Markov chain are given by an exponentially distributed random variable is useful because it gives us an idea for a new type of random variable.  We get this new type of random variable by asking the question "what happens if the amount of times between events is not distributed according to an exponential distributed random variable.  What would happen if the timings between events was given by a different type of random variable?"  Clearly, the resulting stochastic process is no longer be Markovian - it is in fact known as a semi-Markov process because what happens during when an event occurs is Markovian.  The ammount of time between events is not Markovian, however.

<b> In the final part of the exercise I want you to investigate these semi-Markov proceses and to draw graphs of how the number of events changes as a function of time if the timings between events are not Markovian.</b>  How would you determine analytical expressions for the average number of events in these cases?  Also <b> explain why the inhomogeneous poisson process that is explained in this video is not an semi-Markov process </b>.

To start you off with this exercise I have written a code to generate a plot of how the number of events changes as a function of time for a semi-Markov process in which the timings between events is a uniform random variable between 0 and 1 in the cell below. 

In [ ]:
def urandom(Tn) :
    return Tn + random.uniform(0,1)

dynamicplot2( 100, urandom )